Asked ChatGPT how to convert bigrams into a network and it showed me the workflow which I corrected for errors and adapted to my data

In [1]:
from pyspark.sql import SparkSession
from graphframes import *
import sparknlp
from sparknlp.annotator import Tokenizer, PerceptronModel
from sparknlp.base import DocumentAssembler
from pyspark.ml import Pipeline
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.annotator import *
from sparknlp.base import *
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF
from pyspark.ml.clustering import LDA
from pyspark.sql import types as T

spark = SparkSession \
        .builder \
        .appName("network") \
        .getOrCreate()

spark_nlp = sparknlp.start()

24/11/24 20:34:28 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
24/11/24 20:34:28 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
data = spark.read.csv("../1moral_data.csv", header= True).select(["id", "cleaned_text"])
data = data.sample(0.001)
data.show(10)

+------+--------------------+
|    id|        cleaned_text|
+------+--------------------+
|163nsc|i discovered a fe...|
|1wwiwh|every morning i h...|
|204aqo|it seems as thoug...|
|5ircyx|hey hey  im back ...|
|6oybi8|m20 today a girl ...|
|a2z51v|im i just fucking...|
|bihqtw|hey guys so some ...|
|ejp9qr|i stumbled upon t...|
|elnp1n|hello everyone iv...|
|ff73wp|in moments where ...|
+------+--------------------+
only showing top 10 rows



Preprocess and get tokens

In [4]:
english = [
    "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", 
    "because", "been", "before", "being", "below", "between", "both", "but", "by", "can", "cannot", "could", "did", 
    "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", 
    "he", "her", "here", "hers", "herself", "him", "himself", "his", "how", "i", "if", "in", "into", "is", "it", 
    "its", "itself", "let", "me", "more", "most", "must", "my", "myself", "no", "nor", "not", "of", "off", "on", 
    "once", "only", "or", "other", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "some", "such", 
    "than", "that", "the", "their", "theirs", "them", "themselves", "then", "there", "these", "they", "this", "those", 
    "through", "to", "too", "under", "until", "up", "very", "was", "we", "were", "what", "when", "where", "which", 
    "while", "who", "whom", "why", "with", "would", "you", "your", "yours", "yourself", "yourselves", "will", "ll", 
    "re", "ve", "d", "s", "m", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", 
    "s", "t", "u", "v", "w", "x", "y", "z", "many", "us", "ok", "hows", "ive", "ill", "im", "cant", "topics", "topic",
    "discuss", "thoughts", "yo", "thats", "whats", "lets", "nothing", "oh", "omg", 
         "things", "stuff", "yall", "haha", "yes", "no", "wo", "like", 'good', 
         'work', 'got', 'going', 'dont', 'really', 'want', 'make', 'think', 
         'know', 'feel', 'people', 'life', "getting", "lot" "great", "i", "me", 
         "my", "myself", "we", "our", "ours", "ourselves", 
        "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", 
        "himself", "she", "her", "hers", "herself", "it", "its", "itself", 
        "they", "them", "their", "theirs","themselves", "what", "which", "who", 
        "whom", "this", "that", "these", "those", "am", "is", "are", "was", 
        "were", "be", "been", "being", "have", "has", "had", "having", "do", 
        "does", "did", "doing", "will", "would", "should", "can", "could", "may",
        "might", "must", "shall", "ought", "about", "above", "across", "after", 
        "against", "along", "amid", "among", "around", "as", "at", "before", "behind",
        "below", "beneath", "beside", "between", "beyond", "but", "by", 
        "concerning", "considering", "despite", "down", "during", "except", "for",
        "from", "in", "inside", "into", "like", "near", "next", "notwithstanding",
        "of", "off", "on", "onto", "opposite", "out", "outside", "over", "past",
        "regarding", "round", "since", "than", "through", "throughout", "till", 
        "to", "toward", "towards", "under", "underneath", "unlike", "until", "up",
        "upon", "versus", "via", "with", "within", "without", "cant", "cannot", 
        "couldve", "couldnt", "didnt", "doesnt", "dont", "hadnt", "hasnt", 
        "havent", "hed", "hell", "hes", "howd", "howll", "hows", "id", "ill", 
        "im", "ive", "isnt", "itd", "itll", "its", "lets", "mightve", "mustve", 
        "mustnt", "shant", "shed", "shell", "shes", "shouldve", "shouldnt", 
        "thatll", "thats", "thered", "therell", "therere", "theres", "theyd", 
        "theyll", "theyre", "theyve", "wed", "well", "were", "weve", "werent", 
        "whatd", "whatll", "whatre", "whats", "whatve", "whend", "whenll", 
        "whens", "whered", "wherell", "wheres", "whichd", "whichll", "whichre", 
        "whichs", "whod", "wholl", "whore", "whos", "whove", "whyd", "whyll", 
        "whys", "wont", "wouldve", "wouldnt", "youd", "youll", "youre", "youve",
        "f", "m", "because", "go", "lot", "get", "still", "way", "something", "much",
        "thing", "someone", "person", "anything", "goes", "ok", "so", "just", "mostly", 
        "put", "also", "lots", "yet", "ha", "etc", "even", "one", "bye", "take"]

time = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", 
        "sunday", "morning", "noon", "afternoon", "evening", "night", "midnight",
        "dawn", "dusk", "week", "weekend", "weekends","weekly", "today", 
        "yesterday", "tomorrow", "yesterdays", "todays", "mondays", "tuesdays",
        "wednesdays", "thursdays", "fridays", "saturdays", "sundays", "day",
        "everyday", "daily", "workday", 'time', 'month', 'year', 'pm', 'am', "ago",
        "year", "now"]

reddit = ["welcome", "hi", "hello", "sub", "reddit", "thanks", "thank", "maybe",
          "wo30", "mods", "mod", "moderators", "subreddit", "btw", "aw", "aww", 
          "aww", "hey", "hello", "join", "joined", "post", "rselfimprovement", "blah"]

topic_specific = ["self", "improvement", "change", "action",
    'change', 'start', 'goal', 'habit', 'new', 'old', 
    'care', 'world', 'everyone', 'love', 'u', 'right', 'mean', 'matter',
    'best', 'step', 'focus', 'hard', 'small',
    'bad', 'help', 'time', 'problem', 'issue', 'advice',
    'bit', 'experience', 'different',
    'point', 'situation', 'negative', 'control', 'positive',
    'use', 'question', 'idea', 'amp', 'medium', 'hour', 'day', 'minute',
    'aaaaloot', "selfimprovement", "_"]

stopwords = english + time + reddit + topic_specific

In [5]:
documentAssembler = DocumentAssembler()\
     .setInputCol("cleaned_text")\
     .setOutputCol('document')

In [6]:
tokenizer = Tokenizer() \
            .setInputCols(['document'])\
            .setOutputCol('tokenized')

In [7]:
normalizer = Normalizer() \
     .setInputCols(['tokenized']) \
     .setOutputCol('normalized')

In [8]:
from sparknlp.base import Finisher

finisher = Finisher().setInputCols(['normalized'])

In [9]:
my_pipeline = Pipeline(
      stages = [
          documentAssembler,
          tokenizer,
          normalizer,
          finisher
      ])

In [10]:
pipelineModel = my_pipeline.fit(data)
processed_data = pipelineModel.transform(data)
processed_data.show(1)

+------+--------------------+--------------------+
|    id|        cleaned_text| finished_normalized|
+------+--------------------+--------------------+
|163nsc|i discovered a fe...|[i, discovered, a...|
+------+--------------------+--------------------+
only showing top 1 row



Create bigrams

In [11]:
from pyspark.sql import functions as F
from pyspark.sql import types as T

def generate_bigrams(tokens):
    return [(tokens[i], tokens[i+1]) for i in range(len(tokens) - 1) \
           if tokens[i] not in stopwords and tokens[i+1] not in stopwords]
                    
bigram_udf = F.udf(generate_bigrams, T.ArrayType(T.ArrayType(T.StringType())))
bigrams_df = processed_data.withColumn("bigrams", bigram_udf(F.col("finished_normalized")))

bigrams_df.select('bigrams').show(10, truncate=100)

+----------------------------------------------------------------------------------------------------+
|                                                                                             bigrams|
+----------------------------------------------------------------------------------------------------+
|[[low, confidence], [hardly, better], [youtube, video], [unlocked, several], [several, habits], [...|
|                [[drive, home], [home, depending], [traffic, days], [decent, chunk], [car, driving]]|
|[[worst, years], [recall, anyways], [mom, always], [always, told], [pushed, away], [constant, pur...|
|[[another, invitation], [entrench, habits], [nearing, participants], [using, slack], [dedicated, ...|
|[[girl, accused], [totally, changed], [pretty, pathetic], [nerdy, crap], [light, blue], [blue, sa...|
|[[fucking, lazy], [always, see], [brainless, nice], [nice, guy], [shit, memory], [watch, tone], [...|
|[[context, first], [nerdy, kid], [always, kind], [held, back], [dream, s

Create network

In [12]:
edges_df = bigrams_df.select("id", F.explode(F.col("bigrams")).alias("bigram"))
edges_df = edges_df.select(
    F.col("bigram")[0].alias("src"),
    F.col("bigram")[1].alias("dst")
)

In [13]:
edges_df.show(20)

+---------+----------+
|      src|       dst|
+---------+----------+
|      low|confidence|
|   hardly|    better|
|  youtube|     video|
| unlocked|   several|
|  several|    habits|
|    goals|   whether|
|     fold|   laundry|
|    arent|     happy|
|   anyone|      else|
|     else|     happy|
|   direct|       set|
|      set|     goals|
| granular|    always|
|   always|       ask|
|  success|       ask|
|  anyones| happiness|
|    happy|    trying|
|  impress|    others|
|   always|      fail|
|defending|    others|
+---------+----------+
only showing top 20 rows



Create vertices

In [14]:
vertices_df = edges_df.select(F.col("src").alias("id")).union(edges_df.select(F.col("dst").alias("id"))).distinct()
vertices_df.write.mode("overwrite").csv("vertices")

In [15]:
vertices_df.show(10)

+------------------+
|                id|
+------------------+
|rgoodbyedepression|
|             often|
|            taking|
|         partially|
|           achieve|
|             watch|
|           traffic|
|          graduate|
|         basically|
|            highly|
+------------------+
only showing top 10 rows



In [16]:
edges_df = edges_df.distinct()
edges_df.write.mode("overwrite").csv("edges")
edges_df.show(10)

+---------+--------+
|      src|     dst|
+---------+--------+
|  desires|   owned|
|  workout|   often|
|    dream|  school|
| unlocked| several|
|     fail| forgive|
|     thus|creating|
|  traffic|    days|
|brainless|    nice|
|  context|   first|
| absolute|    mess|
+---------+--------+
only showing top 10 rows



In [17]:
network = GraphFrame(vertices_df, edges_df)
network

/software/spark-3.3.2-el8-x86_64/python/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


GraphFrame(v:[id: string], e:[src: string, dst: string])

In [18]:
network.inDegrees.orderBy(F.col("inDegree"), ascending=False).show()

/software/spark-3.3.2-el8-x86_64/python/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


+----------+--------+
|        id|inDegree|
+----------+--------+
|     every|      20|
|    social|      13|
|      back|      11|
|    school|      10|
|    others|      10|
|everything|       9|
|       job|       9|
|      part|       9|
|   friends|       8|
|    better|       8|
|     wrong|       7|
|       try|       7|
|     years|       7|
|    enough|       7|
|     truth|       7|
|     first|       7|
|     hours|       7|
|       ask|       7|
|      stop|       7|
|     human|       6|
+----------+--------+
only showing top 20 rows



In [19]:
results = network.pageRank(resetProbability=0.01, maxIter=20)
results.vertices.select("id", "pagerank").orderBy(F.col("pagerank"), ascending=False).show(50)

+-------------+------------------+
|           id|          pagerank|
+-------------+------------------+
|        femmy| 36.50755241322142|
| distractions| 36.50755241322142|
|        every|15.570693741953018|
|         part|11.166338748531777|
|       school|   9.2506214000977|
|      mindset| 8.822736565721419|
|         stop| 8.197516504777868|
|       social|7.7860415519264485|
|      friends| 6.917299732502962|
|       almost| 6.791626158077601|
|        truth| 6.712792304029796|
|         back| 6.631590769467356|
|        first|  6.50212656297195|
|       months| 6.346217234274218|
|     although| 6.203777764284255|
|   everything| 5.781925694292085|
|        toxic| 5.780099076070397|
|         away| 5.582484758795605|
|        fixed| 5.477750409187343|
|       eating|  5.41253139166865|
|      luckily| 5.221437246518929|
|         joke| 5.141714904134607|
|         else| 5.060567061161978|
|         made| 4.996619243753398|
|       issues|4.9951133546254765|
|       family| 4.97